# Importing Libraries

In [18]:
import pandas as pd
import re
from datetime import timedelta
import sys
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Loading the dataset

In [19]:
df=pd.read_excel("Sentiment_Lysol Aug 2022 ECOM(Scrapped).xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Unnamed: 0           505 non-null    int64         
 1   ID                   505 non-null    int64         
 2   Date                 505 non-null    datetime64[ns]
 3   Month                505 non-null    object        
 4   Review ID            505 non-null    object        
 5   Asin                 0 non-null      float64       
 6   Product Name         505 non-null    object        
 7   Review               505 non-null    object        
 8   Title                505 non-null    object        
 9   Full Text            505 non-null    object        
 10  Translated Text      505 non-null    object        
 11  Brand                505 non-null    object        
 12  Brand Final          505 non-null    object        
 13  Category             0 non-null    

# Data Preprocessing

In [20]:
df['Full Text']=df['Translated Text'].str.lower()

In [21]:
df.head()

Unnamed: 0  ID       Date   Month                             Review ID  \
0           0   1 2022-08-29  August  ae64dece-3337-418f-b9f2-9397e12137b5   
1           1   2 2022-08-21  August  4b3b1628-cf84-4d77-88c6-73a731c1a711   
2           2   3 2022-08-20  August  a58740b5-b85f-4a49-9489-c90fd3b5ada4   
3           3   4 2022-08-01  August  28a7e03e-2e14-4a36-95df-5faddfe28205   
4           4   5 2022-08-28  August  71e41bc7-d4a2-4519-ac60-0afa50d1f329   

   Asin                                       Product Name  \
0   NaN  Clorox Scentiva Disinfecting Wipes - Coconut -...   
1   NaN  Clorox Scentiva Disinfecting Wipes - Coconut -...   
2   NaN  Clorox Scentiva Disinfecting Wipes - Coconut -...   
3   NaN  Clorox Scentiva Disinfecting Wipes - Coconut -...   
4   NaN  Clorox Bleach Free Disinfecting Wipes Value Pa...   

                                              Review  \
0  Uhhhhh why does this scent stick to your hands...   
1     Love the smell! These are My fave scent !!!!!!   
2  These wipes smell so delicious! Makes me looki...   
3  These are AMAZING. As a cleaning lover how did...   
4  It's hard to tell from the picture, but these ...   

                                               Title  \
0                   Can't get the scent off my hands   
1                                        Great smell   
2                             Favorite clorox wipes!   
3                                   My new favorite!   
4  Beware, the 105-wipe count is total for the 3-...   

                                           Full Text  \
0  can't get the scent off my hands uhhhhh why do...   
1  great smell love the smell! these are my fave ...   
2  favorite clorox wipes! these wipes smell so de...   
3  my new favorite! these are amazing. as a clean...   
4  beware, the 105-wipe count is total for the 3-...   

                                     Translated Text   Brand Brand Final  \
0  Can't get the scent off my hands Uhhhhh why do...  Clorox      Clorox   
1  Great smell Love the smell! These are My fave ...  Clorox      Clorox   
2  Favorite clorox wipes! These wipes smell so de...  Clorox      Clorox   
3  My new favorite! These are AMAZING. As a clean...  Clorox      Clorox   
4  Beware, the 105-wipe count is total for the 3-...  Clorox      Clorox   

   Category     Source Site       Origin  Rating Analytics Sentiment  
0       NaN  Target US   US  WebSrapping     3.0            Positive  
1       NaN  Target US   US  WebSrapping     5.0            Positive  
2       NaN  Target US   US  WebSrapping     5.0            Positive  
3       NaN  Target US   US  WebSrapping     5.0            Positive  
4       NaN  Target US   US  WebSrapping     3.0            Negative

In [22]:
df2=df[df['Rating']<=3]

In [23]:
df2.head()

Unnamed: 0   ID       Date   Month                             Review ID  \
0             0    1 2022-08-29  August  ae64dece-3337-418f-b9f2-9397e12137b5   
4             4    5 2022-08-28  August  71e41bc7-d4a2-4519-ac60-0afa50d1f329   
5             5    6 2022-08-28  August  1463f8db-237a-4a6f-a17f-dcc09d13ad8a   
85           85   86 2022-08-09  August  d9210dd5-b3a4-4759-86c3-62784c86ddd8   
150         150  151 2022-08-16  August  8cace2f1-5654-4ad9-9756-59f459b8324b   

     Asin                                       Product Name  \
0     NaN  Clorox Scentiva Disinfecting Wipes - Coconut -...   
4     NaN  Clorox Bleach Free Disinfecting Wipes Value Pa...   
5     NaN  Clorox Bleach Free Disinfecting Wipes Value Pa...   
85    NaN  Clorox Bleach Free Disinfecting Wipes Value Pa...   
150   NaN  Clorox Splash-Less Liquid Bleach - Crisp Lemon...   

                                                Review  \
0    Uhhhhh why does this scent stick to your hands...   
4    It's hard to tell from the picture, but these ...   
5             Product completely damaged upon arrival.   
85   I gave one star because target customer servic...   
150  Doesn't give off a lemon scent. I don't know i...   

                                                 Title  \
0                     Can't get the scent off my hands   
4    Beware, the 105-wipe count is total for the 3-...   
5                                              Damaged   
85                        Misleading target gift money   
150                                              Scent   

                                             Full Text  \
0    can't get the scent off my hands uhhhhh why do...   
4    beware, the 105-wipe count is total for the 3-...   
5     damaged product completely damaged upon arrival.   
85   misleading target gift money i gave one star b...   
150  scent doesn't give off a lemon scent. i don't ...   

                                       Translated Text   Brand Brand Final  \
0    Can't get the scent off my hands Uhhhhh why do...  Clorox      Clorox   
4    Beware, the 105-wipe count is total for the 3-...  Clorox      Clorox   
5     Damaged Product completely damaged upon arrival.  Clorox      Clorox   
85   Misleading target gift money I gave one star b...  Clorox      Clorox   
150  Scent Doesn't give off a lemon scent. I don't ...  Clorox      Clorox   

     Category     Source Site       Origin  Rating Analytics Sentiment  
0         NaN  Target US   US  WebSrapping     3.0            Positive  
4         NaN  Target US   US  WebSrapping     3.0            Negative  
5         NaN  Target US   US  WebSrapping     1.0            Negative  
85        NaN  Target US   US  WebSrapping     1.0            Negative  
150       NaN  Target US   US  WebSrapping     2.0            Negative

In [24]:
# Negative
for index, row in df2.iterrows(): 
        if len((re.findall(r'(bad|poor|waste|not good|not as good|not worth|horrible|disgust|did not like it|unhappy|terrible|useless|worst|very bad|badly|just bad|disappoint|useless|not satisfied|satisfied|cheap quality|horrendous|disgusting|disgust|not happy)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="General Complaints"
            df2.loc[index,'Level2']="General Complaints"
            df2.loc[index,'Level1']="Product Perception"
        if len((re.findall(r'(counterfeit|fake|knockoff|fraud|chinese|china|duplicate|false advertisement|scam|advertised|spam)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Fake"
            df2.loc[index,'Level2']="Claims Of Counterfeit Product"
            df2.loc[index,'Level1']="Product Perception"
        
            
            
        if len((re.findall(r'(dirt|effective|ineffective|Leaves Spots/Stains/Streaks|Ineffective Cleaning|less effective|dirty|clean|unclean|not clean|clean|harder to be clean|never clean|clean poorly|do not come out clean|woudnt use to clean|not getting clean)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Ineffective cleaning"
            df2.loc[index,'Level2']="Ineffective Cleaning"
            df2.loc[index,'Level1']="Functionality"
        if len((re.findall(r'(disinfect|germs|ineffective disinfectant|corona|covid|virus|bacteria|bad disinfectant)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Ineffective Disinfectant"
            df2.loc[index,'Level2']="Ineffective Disinfectant"
            df2.loc[index,'Level1']="Functionality"
        if len((re.findall(r'(residue|film|Leaves Residue)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Leaves Residue"
            df2.loc[index,'Level2']="Ineffective Cleaning"
            df2.loc[index,'Level1']="Functionality"
        if len((re.findall(r'(stain)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Does Not Remove Stains"
            df2.loc[index,'Level2']="Ineffective Cleaning"
            df2.loc[index,'Level1']="Functionality"
        if len((re.findall(r'(foam)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Excess Foam"
            df2.loc[index,'Level2']="Ineffective Cleaning"
            df2.loc[index,'Level1']="Functionality"
        if len((re.findall(r'(pigment)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Leave Pigment"
            df2.loc[index,'Level2']="Ineffective Cleaning"
            df2.loc[index,'Level1']="Functionality"
            
            
        if len((re.findall(r'(late delivery|expired product|used product|missing product|dislike packaging|misplaced/wrong product|delivery man|delivery guy|delivery issue|delivery|delivered|deliver|wrong delivery|delivered|did not deliver)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Late Delivery"
            df2.loc[index,'Level2']="Delivery Issues"
            df2.loc[index,'Level1']="Shipping Experience"
        if len((re.findall(r'(damage|damaged|smashed|falling apart|cracked|torn|container damaged)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Damaged Product"
            df2.loc[index,'Level2']="Damaged/Unsealed/Leaked Products"
            df2.loc[index,'Level1']="Shipping Experience"
        if len((re.findall(r'(broken|broke|crush|crushed|crushed/broken product|damaged product|unsealed product|leaked product)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Crushed/Broken Product"
            df2.loc[index,'Level2']="Damaged/Unsealed/Leaked Products"
            df2.loc[index,'Level1']="Shipping Experience"
        if len((re.findall(r'(leak|leaking|leaked|spill|spilled|soak|soaked|dripping|dripped|wet|flow out)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Leaked Product"
            df2.loc[index,'Level2']="Damaged/Unsealed/Leaked Products"
            df2.loc[index,'Level1']="Shipping Experience"
        if len((re.findall(r'(missing|half empty|empty)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Missing Product"
            df2.loc[index,'Level2']="Delivery Issues"
            df2.loc[index,'Level1']="Shipping Experience"
        if len((re.findall(r'(expire)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Expired Product"
            df2.loc[index,'Level2']="Delivery Issues"
            df2.loc[index,'Level1']="Shipping Experience"
        if len((re.findall(r'(unsealed|seal|not seal|open)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Unsealed Product"
            df2.loc[index,'Level2']="Damaged/Unsealed/Leaked Products"
            df2.loc[index,'Level1']="Shipping Experience"
        if len((re.findall(r'(unsealed|seal|not seal|open)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Unsealed Product"
            df2.loc[index,'Level2']="Damaged/Unsealed/Leaked Products"
            df2.loc[index,'Level1']="Shipping Experience"    
            
            
            
        if len((re.findall(r'(defect|defective|poor quality|low quality|non-durable|falls apart|falling apart|fell apart|falling off|fall apart)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Defective Product"
            df2.loc[index,'Level2']="Defective/Poor Quality/Non-Durable Product"
            df2.loc[index,'Level1']="Manufacturing & Design"
        if len((re.findall(r'(size|design|thinner|thick|hard|dislike composition|dryness|not last very long|does not last even a week|dislike thickness|dislike texture|dislike size|dislike quantity|dislike design|dislike consistency|thicker|diluted|texture|too wet|too dry)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Size/Design"
            df2.loc[index,'Level2']="Size/Design"
            df2.loc[index,'Level1']="Manufacturing & Design"
        
        
             
        if len((re.findall(r'(smell|scent|no smell|bad smell|smell longevity issue|fragrance|odor|pungent|smelled|fragrant)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Unpleasant Fragrance"
            df2.loc[index,'Level2']="Dislike Fragrance"
            df2.loc[index,'Level1']="Product Features"
        if len((re.findall(r'(strong smell|strong fragrance)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Strong Fragrance"
            df2.loc[index,'Level2']="Dislike Fragrance"
            df2.loc[index,'Level1']="Product Features"
            
            
        if len((re.findall(r'(breathe|asthma|allergy|burning issue|skin harm|respiratory issue|headache|cough|dry skin|choking|child harm|health|rash|itching|itch|cancer|burn|burned|sinus|headache|dry skin)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Personal Harm"
            df2.loc[index,'Level2']="Medical Issues"
            df2.loc[index,'Level1']="Adverse Event"
        if len((re.findall(r'(pet harm)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Pet Harm"
            df2.loc[index,'Level2']="Pet Harm"
            df2.loc[index,'Level1']="Adverse Event"
        if len((re.findall(r'(property damage)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Damage Laundry"
            df2.loc[index,'Level2']="Property Damage"
            df2.loc[index,'Level1']="Adverse Event" 
        if len((re.findall(r'(chemicals|poisonous)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Chemicals"
            df2.loc[index,'Level2']="toxic"
            df2.loc[index,'Level1']="Adverse Event"    
                
            
            
            
        if len((re.findall(r'(expensive|costly|ripoff|pricey|not cheap|price|money|cost)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Expensive"
            df2.loc[index,'Level2']="Expensive"
            df2.loc[index,'Level1']="Pricing"
            
            
            
        if len((re.findall(r'(difficult|inconvenient|not easy to use)',str(df2.loc[index,'Translated Text']))))>0:
            df2.loc[index,'Level3']="Difficult to Use"
            df2.loc[index,'Level2']="Difficult to Use"
            df2.loc[index,'Level1']="Usability"
            
            
        
        

C:\Users\ujjwal.pratik1\AppData\Local\Temp\ipykernel_26572\838997514.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[index,'Level3']="General Complaints"
C:\Users\ujjwal.pratik1\AppData\Local\Temp\ipykernel_26572\838997514.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[index,'Level2']="General Complaints"
C:\Users\ujjwal.pratik1\AppData\Local\Temp\ipykernel_26572\838997514.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [25]:
pd.crosstab(index=df2["Level1"],columns="count") 

col_0                   count
Level1                       
Adverse Event               2
Functionality               4
Manufacturing & Design      9
Pricing                     6
Product Features           10
Product Perception          1
Shipping Experience         4

In [26]:
pd.crosstab(index=df2["Level2"],columns="count") 

col_0                                       count
Level2                                           
Damaged/Unsealed/Leaked Products                4
Defective/Poor Quality/Non-Durable Product      3
Dislike Fragrance                              10
Expensive                                       6
General Complaints                              1
Ineffective Cleaning                            4
Medical Issues                                  2
Size/Design                                     6

In [27]:
df2.isnull().sum()

Unnamed: 0              0
ID                      0
Date                    0
Month                   0
Review ID               0
Asin                   52
Product Name            0
Review                  0
Title                   0
Full Text               0
Translated Text         0
Brand                   0
Brand Final             0
Category               52
Source                  0
Site                    0
Origin                  0
Rating                  0
Analytics Sentiment     0
Level3                 16
Level2                 16
Level1                 16
dtype: int64

In [28]:
df3=df[df['Rating']>3]

In [29]:
df3.head()

Unnamed: 0  ID       Date   Month                             Review ID  \
1           1   2 2022-08-21  August  4b3b1628-cf84-4d77-88c6-73a731c1a711   
2           2   3 2022-08-20  August  a58740b5-b85f-4a49-9489-c90fd3b5ada4   
3           3   4 2022-08-01  August  28a7e03e-2e14-4a36-95df-5faddfe28205   
6           6   7 2022-08-18  August  bab3d860-c537-4965-a55c-50d46ca141b3   
7           7   8 2022-08-18  August  f758cdfb-8efc-4d6e-a193-a5f700d58c28   

   Asin                                       Product Name  \
1   NaN  Clorox Scentiva Disinfecting Wipes - Coconut -...   
2   NaN  Clorox Scentiva Disinfecting Wipes - Coconut -...   
3   NaN  Clorox Scentiva Disinfecting Wipes - Coconut -...   
6   NaN  Clorox Bleach Free Disinfecting Wipes Value Pa...   
7   NaN  Clorox Bleach Free Disinfecting Wipes Value Pa...   

                                              Review                   Title  \
1     Love the smell! These are My fave scent !!!!!!             Great smell   
2  These wipes smell so delicious! Makes me looki...  Favorite clorox wipes!   
3  These are AMAZING. As a cleaning lover how did...        My new favorite!   
6  Spent a lot of time with these wipes during th...                   Wipes   
7  Love these wipes. For a quick clean up or wipi...                So handy   

                                           Full Text  \
1  great smell love the smell! these are my fave ...   
2  favorite clorox wipes! these wipes smell so de...   
3  my new favorite! these are amazing. as a clean...   
6  wipes spent a lot of time with these wipes dur...   
7  so handy love these wipes. for a quick clean u...   

                                     Translated Text   Brand Brand Final  \
1  Great smell Love the smell! These are My fave ...  Clorox      Clorox   
2  Favorite clorox wipes! These wipes smell so de...  Clorox      Clorox   
3  My new favorite! These are AMAZING. As a clean...  Clorox      Clorox   
6  Wipes Spent a lot of time with these wipes dur...  Clorox      Clorox   
7  So handy Love these wipes. For a quick clean u...  Clorox      Clorox   

   Category     Source Site       Origin  Rating Analytics Sentiment  
1       NaN  Target US   US  WebSrapping     5.0            Positive  
2       NaN  Target US   US  WebSrapping     5.0            Positive  
3       NaN  Target US   US  WebSrapping     5.0            Positive  
6       NaN  Target US   US  WebSrapping     5.0            Positive  
7       NaN  Target US   US  WebSrapping     5.0            Positive

In [30]:
#df3['Level3']=" "
#df3['Level1']=" "
# Positive
for index, row in df3.iterrows(): 
        if len((re.findall(r'(good|ok|okay|All Purpose|one|decent|Decent|awesome|must have|amazing|well|work|thank|reliable|cool|satisfied|as promised|satisfy|need|favourite|favorite|helpful|good product|as described|does the job|exactly|no negative|buy again|just as described|what i needed|praise|undamaged|not bad|like|work well|works|works well|good stuff|greatful|perfect|fine|great|happy|glad|love|good|perfect|nice|excellent|ideal|best)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Positive Feedback"
            df3.loc[index,'Level2']="Positive Feedback"
            df3.loc[index,'Level1']="Product Perception"
        if len((re.findall(r'(years|many years|year|long)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Product Appreciation"
            df3.loc[index,'Level2']="Product Appreciation"
            df3.loc[index,'Level1']="Product Perception"
        if len((re.findall(r'(Sustainability|sustainable)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="sustainability"
            df3.loc[index,'Level2']="sustainability"
            df3.loc[index,'Level1']="Product Perception"
            
            
            
        if len((re.findall(r'(good quality|quality|damage|leak|broken|damaged|damaged|package|packaged|good packaging|nice packaging)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Appreciating Packaging"
            df3.loc[index,'Level2']="Quick Delivery/Appreciate Packaging"
            df3.loc[index,'Level1']="Shipping Experience"
        if len((re.findall(r'(delivery|delivered|fast delivery|quick|on time|delivery|great delivery|delivered|timely delivery|early deliver|delivered|arrived|arrive)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Quick Delivery"
            df3.loc[index,'Level2']="Quick Delivery/Appreciate Packaging"
            df3.loc[index,'Level1']="Shipping Experience"
            
            
        
        if len((re.findall(r'(clean|spot|stain|stains|job|effective|works|work|work well|work good|works well|works good|sparkle|sparkling|cleans well|clean very well|hard water|clear|glasses clear|residue|spot|sparkling clean|look clean|super clean|bowl clean|great work|clean extremely|sparkle|shine|sparkling|effective|efficient|works|work|job)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Effective cleaning"
            df3.loc[index,'Level2']="Effective Cleaning"
            df3.loc[index,'Level1']="Functionality"
        if len((re.findall(r'(spot|stain|spots|streak free)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Spotless/Stain less"
            df3.loc[index,'Level2']="Effective Cleaning"
            df3.loc[index,'Level1']="Functionality"
        if len((re.findall(r'(disinfect|germs|germ|ineffective disinfectant|corona|covid|virus|bacteria|bad disinfectant)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Effective Disinfectant"
            df3.loc[index,'Level2']="Effective Disinfectant"
            df3.loc[index,'Level1']="Functionality"
            
            
            
            
        if len((re.findall(r'(smell|fragrance|freshness|fresh|scent|odor|odors|aroma|smelled|fragrant|aromatic|smelled|smelling|fragrant)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Pleasant Fragrance"
            df3.loc[index,'Level2']="Pleasant Fragrance"
            df3.loc[index,'Level1']="Product Features"
        if len((re.findall(r'(Lemon)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Lemon Fragrance"
            df3.loc[index,'Level2']="Pleasant Fragrance"
            df3.loc[index,'Level1']="Product Features"
            
            
        if len((re.findall(r'(money|cheap|price|worth|cost|cheaper|value for money|good value|affordable|buy|deal|value)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Value for Money"
            df3.loc[index,'Level2']="Value for Money"
            df3.loc[index,'Level1']="Pricing"
        if len((re.findall(r'(affordable|cheap)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Affordable"
            df3.loc[index,'Level2']="Value for Money"
            df3.loc[index,'Level1']="Pricing"
        if len((re.findall(r'(cost effective|cost efficient)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Cost Effective"
            df3.loc[index,'Level2']="Value for Money"
            df3.loc[index,'Level1']="Pricing"
        if len((re.findall(r'(deal)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Good Deal"
            df3.loc[index,'Level2']="Value for Money"
            df3.loc[index,'Level1']="Pricing"
        if len((re.findall(r'(worth)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Worth the Money"
            df3.loc[index,'Level2']="Value for Money"
            df3.loc[index,'Level1']="Pricing"
            
            
        if len((re.findall(r'(easy to use|ease to use|easy|easy to take|convenient|useful|handy)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Easy to Use"
            df3.loc[index,'Level2']="Easy to Use"
            df3.loc[index,'Level1']="Usability"
            
            
       
        if len((re.findall(r'(good/nice size|good/nice design|like consistency|like quantity|like sizelike design|thinner|thick|hard|thicker|diluted|nice/good texture|durable)',str(df3.loc[index,'Translated Text']))))>0:
            df3.loc[index,'Level3']="Size/Design"
            df3.loc[index,'Level2']="Size/Design"
            df3.loc[index,'Level1']="Manufacturing & Design"
            
            
            
       # if len((re.findall(r'(counterfeit|fake|knockoff|fraud|chinese|china|duplicate|false advertisement|scam|advertised)',str(df3.loc[index,'Translated Text']))))>0:
            # df3.loc[index,'Level3']="Fake"
            #df3.loc[index,'Level2']="Claims Of Counterfeit Product"
            # df3.loc[index,'Level1']="Product Perception"
        

C:\Users\ujjwal.pratik1\AppData\Local\Temp\ipykernel_26572\80155163.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.loc[index,'Level3']="Pleasant Fragrance"
C:\Users\ujjwal.pratik1\AppData\Local\Temp\ipykernel_26572\80155163.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.loc[index,'Level2']="Pleasant Fragrance"
C:\Users\ujjwal.pratik1\AppData\Local\Temp\ipykernel_26572\80155163.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [31]:
pd.crosstab(index=df3["Level1"],columns="count") 

col_0                   count
Level1                       
Functionality              89
Manufacturing & Design     16
Pricing                    44
Product Features           59
Product Perception         32
Shipping Experience        13
Usability                 111

In [32]:
pd.crosstab(index=df3["Level1"],columns="count") 

col_0                   count
Level1                       
Functionality              89
Manufacturing & Design     16
Pricing                    44
Product Features           59
Product Perception         32
Shipping Experience        13
Usability                 111

In [33]:
df3.isnull().sum()

Unnamed: 0               0
ID                       0
Date                     0
Month                    0
Review ID                0
Asin                   443
Product Name             0
Review                   0
Title                    0
Full Text                0
Translated Text          0
Brand                    0
Brand Final              0
Category               443
Source                   0
Site                     0
Origin                   0
Rating                   0
Analytics Sentiment      0
Level3                  79
Level2                  79
Level1                  79
dtype: int64

In [34]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 443 entries, 1 to 504
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Unnamed: 0           443 non-null    int64         
 1   ID                   443 non-null    int64         
 2   Date                 443 non-null    datetime64[ns]
 3   Month                443 non-null    object        
 4   Review ID            443 non-null    object        
 5   Asin                 0 non-null      float64       
 6   Product Name         443 non-null    object        
 7   Review               443 non-null    object        
 8   Title                443 non-null    object        
 9   Full Text            443 non-null    object        
 10  Translated Text      443 non-null    object        
 11  Brand                443 non-null    object        
 12  Brand Final          443 non-null    object        
 13  Category             0 non-null    

In [35]:
df4=pd.concat([df2,df3], ignore_index=True)

In [36]:
df4.isnull().sum()

Unnamed: 0               0
ID                       0
Date                     0
Month                    0
Review ID                0
Asin                   495
Product Name             0
Review                   0
Title                    0
Full Text                0
Translated Text          0
Brand                    0
Brand Final              0
Category               495
Source                   0
Site                     0
Origin                   0
Rating                   0
Analytics Sentiment      0
Level3                  95
Level2                  95
Level1                  95
dtype: int64

In [37]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Unnamed: 0           495 non-null    int64         
 1   ID                   495 non-null    int64         
 2   Date                 495 non-null    datetime64[ns]
 3   Month                495 non-null    object        
 4   Review ID            495 non-null    object        
 5   Asin                 0 non-null      float64       
 6   Product Name         495 non-null    object        
 7   Review               495 non-null    object        
 8   Title                495 non-null    object        
 9   Full Text            495 non-null    object        
 10  Translated Text      495 non-null    object        
 11  Brand                495 non-null    object        
 12  Brand Final          495 non-null    object        
 13  Category             0 non-null    

In [38]:
#df5=df4[df4['Level1'].isnull()]

In [39]:
#df5.to_excel('null_lysol.xlsx')

In [43]:
df4.to_excel('Sentiment_keyword_Lysol Aug 2022 ECOM(Scrapped).xlsx')

In [41]:
df4.isnull().sum()

Unnamed: 0               0
ID                       0
Date                     0
Month                    0
Review ID                0
Asin                   495
Product Name             0
Review                   0
Title                    0
Full Text                0
Translated Text          0
Brand                    0
Brand Final              0
Category               495
Source                   0
Site                     0
Origin                   0
Rating                   0
Analytics Sentiment      0
Level3                  95
Level2                  95
Level1                  95
dtype: int64

In [42]:
df4

Unnamed: 0   ID       Date   Month                             Review ID  \
0             0    1 2022-08-29  August  ae64dece-3337-418f-b9f2-9397e12137b5   
1             4    5 2022-08-28  August  71e41bc7-d4a2-4519-ac60-0afa50d1f329   
2             5    6 2022-08-28  August  1463f8db-237a-4a6f-a17f-dcc09d13ad8a   
3            85   86 2022-08-09  August  d9210dd5-b3a4-4759-86c3-62784c86ddd8   
4           150  151 2022-08-16  August  8cace2f1-5654-4ad9-9756-59f459b8324b   
..          ...  ...        ...     ...                                   ...   
490         499  501 2022-08-03  August  c303307d-5993-4bae-bf7a-e7d73468097a   
491         501  503 2022-08-02  August  8a714938-c751-4503-9699-f799ee3f394f   
492         502  504 2022-08-09  August  ddd038b5-a601-45ae-8c9b-520867aaeced   
493         503  505 2022-08-20  August  3a65c39b-97d4-4ca4-a86c-c11bc0404361   
494         504  506 2022-08-24  August  52a4d0ef-2bd7-45da-be85-cba8fb01d461   

     Asin                                       Product Name  \
0     NaN  Clorox Scentiva Disinfecting Wipes - Coconut -...   
1     NaN  Clorox Bleach Free Disinfecting Wipes Value Pa...   
2     NaN  Clorox Bleach Free Disinfecting Wipes Value Pa...   
3     NaN  Clorox Bleach Free Disinfecting Wipes Value Pa...   
4     NaN  Clorox Splash-Less Liquid Bleach - Crisp Lemon...   
..    ...                                                ...   
490   NaN      Swiffer WetJet Wood Mopping Pad Refill - 12ct   
491   NaN  Swiffer WetJet Heavy Duty Mopping Pads Refill ...   
492   NaN  Swiffer Sweeper X-Large Wet Mopping Pad Multi ...   
493   NaN  Swiffer Sweeper Wet Mopping Cloth - Lavender -...   
494   NaN             Swiffer Sweeper Wet Refill Wood - 20ct   

                                                Review  \
0    Uhhhhh why does this scent stick to your hands...   
1    It's hard to tell from the picture, but these ...   
2             Product completely damaged upon arrival.   
3    I gave one star because target customer servic...   
4    Doesn't give off a lemon scent. I don't know i...   
..                                                 ...   
490  The pads are easy to switch and hold a good am...   
491  These are stronger than the original and you c...   
492  I like using these to collect dirt before I mo...   
493  nice long lasting subtle clean room smell afte...   
494         Cleans great! Easy to use. Great packaging   

                                                 Title  \
0                     Can't get the scent off my hands   
1    Beware, the 105-wipe count is total for the 3-...   
2                                              Damaged   
3                         Misleading target gift money   
4                                                Scent   
..                                                 ...   
490                                Wet Jet Wood Review   
491                                       Just better!   
492                            Good for light cleaning   
493                       very good cleaning and smell   
494                                              Great   

                                             Full Text  ... Brand Final  \
0    can't get the scent off my hands uhhhhh why do...  ...      Clorox   
1    beware, the 105-wipe count is total for the 3-...  ...      Clorox   
2     damaged product completely damaged upon arrival.  ...      Clorox   
3    misleading target gift money i gave one star b...  ...      Clorox   
4    scent doesn't give off a lemon scent. i don't ...  ...      Clorox   
..                                                 ...  ...         ...   
490  wet jet wood reviewthe pads are easy to switch...  ...     Swiffer   
491  just better!these are stronger than the origin...  ...     Swiffer   
492  good for light cleaningi like using these to c...  ...     Swiffer   
493  very good cleaning and smellnice long lasting ...  ...     Swiffer   
494    greatcleans great! easy to use. great